In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from random import randint
import shutil

## 추가사항
이전 딥러닝 소스코드에서 가중치 초기화시 0혹은 0~1사이 랜덤값을 부여한 반면 이번에는 가중치의 초기값을 의미 있는 값으로 초기화 하여 학습을 원활하게 해준다.

## MNIST자료 불러오기
28*28사이즈의 숫자 이미지를 행렬로 표현하였으며 2차원 구조를 정규화 하여 1차원 구조로 표현되있다.
라벨의 경우 one-hot방식인데 0~9까지의 숫자를 [0,0,0,0,0,1,0,0,0,0]이런 식으로 표현하였다.

In [2]:
mnist = input_data.read_data_sets("./MNIST_data/",one_hot=True)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
print("총 학습용 데이터 갯수 : ",len(mnist.train.labels))
print("총 테스트용 데이터 갯수 : ",len(mnist.test.labels))
print("총 검증용 데이터 갯수 : ",len(mnist.validation.labels))
print("개별 데이터의 행렬 길이 : ",len(mnist.train.images[0]))
print("개별 라벨의 행렬 길이 : ", len(mnist.train.labels[0]))

총 학습용 데이터 갯수 :  55000
총 테스트용 데이터 갯수 :  10000
총 검증용 데이터 갯수 :  5000
개별 데이터의 행렬 길이 :  784
개별 라벨의 행렬 길이 :  10


## 가중치 초기화
0이 아닌 값으로 가중치를 초기해 줌으로써 원활한 학습을 할수 있게 보조한다.

In [4]:
def xaver_init(n_inputs, n_outputs, uniform = True):
    if uniform:
        init_range = tf.sqrt(6.0/ (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)

    else:
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

## 파라미터 설정

In [5]:
x = tf.placeholder(tf.float32,[None,784], name="images")
label = tf.placeholder(tf.float32,[None,10],name="labels")

W1 = tf.get_variable("W1", shape=[784,256], initializer=xaver_init(784,256))
W2 = tf.get_variable("W2", shape=[256,256], initializer=xaver_init(256,256))
W3 = tf.get_variable("W3", shape=[256,10], initializer=xaver_init(256,10))

b1 = tf.Variable(tf.zeros([256]),name="bias1")
b2 = tf.Variable(tf.zeros([256]),name="bias2")
b3 = tf.Variable(tf.zeros([10]),name="bias3")

## 학습설정 파라미터

In [6]:
learning_rate = 0.005
# 전체 학습 횟수
training_epochs = 15
# 단일 학습에 사용할 자료량
batch_size = 100
# 학습 결과 노출 간격
display_step = 1
# 전체 학습 자료를 단일 반복당 학습 자료수로 나누어 한 학습당 반복횟수 산정
total_batch = int(mnist.train.num_examples/batch_size)
# 텐서보드용 로그 저장 경로
log_path = "./logs/mnist4deep_2"

## 수치 요약

In [7]:
W1_hist = tf.histogram_summary("Weights1",W1)
W2_hist = tf.histogram_summary("Weights2",W2)
W3_hist = tf.histogram_summary("Weights3",W3)

b1_hist = tf.histogram_summary("biases1",b1)
b2_hist = tf.histogram_summary("biases2",b2)
b3_hist = tf.histogram_summary("biases3",b3)

label_hist = tf.histogram_summary("labels",label)

## 가설

In [8]:
with tf.name_scope("Construct_model") as scope:
    with tf.name_scope("L1") as scope:
        L1 = tf.nn.relu(tf.add(tf.matmul(x,W1),b1))
    with tf.name_scope("L2") as scope:
        L2 = tf.nn.relu(tf.add(tf.matmul(L1,W2),b2))
    with tf.name_scope("L-hypothesis") as scope:
        hypothesis  = tf.add(tf.matmul(L2,W3),b3)

## 비용함수(교차엔트로피)

In [9]:
with tf.name_scope("cost_func") as scope:
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(hypothesis,label))
    cost_summary = tf.scalar_summary("cost",cost)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

## 모델 평가

In [10]:
with tf.name_scope("test") as scope:
    real_label = tf.arg_max(label,1) # 실제 라벨
    learn_label = tf.arg_max(hypothesis,1) # 머신러닝 결과
    correct_prodiction = tf.equal(real_label,learn_label) # 둘다 값이 같으면 true, 틀리면 false

    # 정확도
    accuracy = tf.reduce_mean(tf.cast(correct_prodiction,tf.float32))
    accuracy_summary = tf.scalar_summary("accuracy",accuracy)

## 변수 초기화 

In [11]:
init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


## 세션

In [12]:
sess = tf.Session()
sess.run(init)

## 텐서보드 설정

In [13]:
merged = tf.merge_all_summaries()
try: # 이전 텐서보드 학습로그 삭제
    shutil.rmtree(log_path)
except:
    pass
writer = tf.train.SummaryWriter(log_path,sess.graph)

## 학습하기

In [14]:
try:
    train_count = 0
    for epoch in range(training_epochs):
        avg_cost=0
        for i in range(total_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            _,c,cs = sess.run([optimizer,cost,cost_summary],feed_dict={x:batch_xs,label:batch_ys})
            avg_cost = c/total_batch
            train_count += 1
            writer.add_summary(cs,train_count) # 매 batch_size만큼 학습 할때마다 cost를 텐서보드에 그림
        print("{}번 학습 cost: {}".format(epoch+1,avg_cost))
        if (epoch+1) % display_step == 0:
            feed = {x:mnist.test.images, label: mnist.test.labels}
            summary,_ = sess.run([merged,accuracy],feed_dict=feed)
            writer.add_summary(summary,epoch)
except Exception as e:
    print(e)
    sess.close()
    exit()
print("학습완료")

1번 학습 cost: 0.00046951028433713047
2번 학습 cost: 0.0003807091442021457
3번 학습 cost: 0.0001055341823534532
4번 학습 cost: 0.000187186368487098
5번 학습 cost: 0.00015099766579541293
6번 학습 cost: 3.046064552935687e-05
7번 학습 cost: 6.604893641038374e-05
8번 학습 cost: 6.880489939993078e-05
9번 학습 cost: 7.855831222100692e-05
10번 학습 cost: 3.179511224681681e-05
11번 학습 cost: 0.0004031111164526506
12번 학습 cost: 8.330173282460732e-06
13번 학습 cost: 1.3749158348549496e-05
14번 학습 cost: 0.0002434243397279219
15번 학습 cost: 0.00015230616385286503
학습완료


## 테스트 데이터로 검증

In [15]:
print(sess.run(accuracy, feed_dict={x: mnist.test.images, label:mnist.test.labels}))
sess.close()

0.9754
